# MIS High level

In [ ]:
import networkx as nx

# Create a simple triangle graph
graph = nx.Graph()
graph.add_edges_from([(0, 1), (0, 2)])

In [ ]:

from mis.shared.types import MISInstance
from mis.pipeline.config import SolverConfig
from mis import MISSolver

Solve using the configuration without quantum

In [ ]:

# Define classical solver configuration
from mis.shared.types import MethodType


config = SolverConfig(
    method = MethodType.EAGER,
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)


# Run the solver
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].energy)

Solve using the configuration with QTip

In [ ]:
from mis.pipeline.backends import QutipBackend


config = SolverConfig(
    method = MethodType.EAGER,
    backend = QutipBackend(),
    max_iterations=1
)

# Create the MIS instance
instance = MISInstance(graph)


# Run the solver
solver = MISSolver(instance, config)
solutions = solver.solve().result()

# Display results
print("MIS solution:", solutions[0].nodes)
print("Solution cost:", solutions[0].energy)

Solve using Remote QPU backend 

In [ ]:
from mis.pipeline.backends import RemoteQPUBackend

USERNAME="username"
PROJECT_ID="123"
PASSWORD=None

if PASSWORD is not None:
    config = SolverConfig(
        method = MethodType.EAGER,
        backend = RemoteQPUBackend(
            username=USERNAME,
            project_id=PROJECT_ID,
            password=PASSWORD
        ),
        max_iterations=1
    )

    # Create the MIS instance
    instance = MISInstance(graph)

    # Run the solver
    solver = MISSolver(instance, config)
    solutions = solver.solve().result()

    # Display results
    print("MIS solution:", solutions[0].nodes)
    print("Solution cost:", solutions[0].energy)